Make sure Python and Jupyter are set up locally first (and if necessary relaunch VSCode). There is a `shell.nix` that does this for you if you have Nix installed, or you can use a virtual env created from the global Python.

```bash
$ python3 -m venv .venv
$ . .venv/bin/activate
$ pip install jupyter ipykernel notebook
$ pip install -r requirements.txt
```

Some LLM code copied from [Qwak.com](https://www.qwak.com/post/utilizing-llms-with-embedding-stores)...

In [1]:
from datasets import load_dataset

# Load only the training split of the dataset
train_dataset = load_dataset("databricks/databricks-dolly-15k", split='train')

# Filter the dataset to only include entries with the 'closed_qa' category
closed_qa_dataset = train_dataset.filter(lambda example: example['category'] == 'closed_qa')

print(closed_qa_dataset[0])

{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


In [2]:
import chromadb
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="knowledge-base")

In [4]:
# Method to populate the vector store with embeddings from a dataset
def populate_vectors(dataset):
    for i, item in enumerate(dataset):
        combined_text = item['context']
        embeddings = embedding_model.encode(combined_text).tolist()
        collection.add(embeddings=[embeddings], documents=[item['context']], ids=[f"id_{i}"])

# Method to search the ChromaDB collection for relevant context based on a query
def search_context(query, n_results=1):
    query_embeddings = embedding_model.encode(query).tolist()
    return collection.query(query_embeddings=query_embeddings, n_results=n_results)

In [5]:
populate_vectors(closed_qa_dataset)

In [5]:
closed_qa_dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 1773
})

In [6]:
collection.get_model()

Collection(id=UUID('d1ffdc0c-5f83-4c1b-b9ad-f9b0bd1adcb4'), name='knowledge-base', configuration_json={'hnsw_configuration': {'space': 'l2', 'ef_construction': 100, 'ef_search': 10, 'num_threads': 8, 'M': 16, 'resize_factor': 1.2, 'batch_size': 100, 'sync_threshold': 1000, '_type': 'HNSWConfigurationInternal'}, '_type': 'CollectionConfigurationInternal'}, metadata=None, dimension=None, tenant='default_tenant', database='default_database', version=0)

In [26]:
user_question = "When was Tomoaki Komorida born?"
context_response = search_context(user_question)
context = "".join(context_response['documents'][0])
context

'Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer. In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Roasso Kumamoto) based in his local region. He played as a regular player and the club was promoted to J2 in 2008. Although he did not play as much, he still played in many matches. In 2010, he moved to Indonesia a

In [19]:
prompt = f"""
{user_question}

Context information is below.
---------------------
{context}
---------------------
Given the context and provided history information and not prior knowledge,
reply to the user comment. If the answer is not in the context, inform
the user that you can't answer the question.
"""

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Make sure ollama is running in the background and the model is available:

```bash
$ ollama serve
$ ollama pull mistral
```

In [30]:
import ollama
response = ollama.chat(model="mistral", messages=[{'role':'user', 'content': prompt}])
response['message']['content']

' Tomoaki Komorida was born on July 10, 1981.'

In [13]:
%%bash
pip install pipreqs
jupyter nbconvert --to=python README.ipynb
pipreqs ==ignore ".venv" .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 11.0 MB/s eta 0:00:00m eta 0:00:010:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=0d029a8ff4638c40fa356fa0811eb9062726c077827fea0c117d5162219a1c1b
  Stored in directory: /home/dsyer/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
  Attempting uninstall: ipython
    Found existing installation: ipython 8.26.0
    Uninstalling ipython-8.26.0:
      Successfully uninstalled ipython-8.26.0

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
pd.read_json("data/lines.jsonl", lines=True)

,instruction,context,response
0,How to make a cup of tea?,,Boil water. Add tea bag. Pour water into cup. ...
1,What is the capital of France?,,Paris
2,What is the capital of Germany?,,Berlin
3,What is the capital of Italy?,,Rome
4,What is the capital of Spain?,,Madrid
5,What is the capital of Portugal?,,Lisbon
6,What is the capital of Greece?,,Athens
7,What is the capital of Turkey?,,Ankara
8,What is the capital of Egypt?,,Cairo
9,What is the capital of South Africa?,,Pretoria


In [10]:
data = load_dataset("./data", split="train")

In [12]:
data.__class__


datasets.arrow_dataset.Dataset